# Agent Deep Q-Network (DQN) dans un environnement GridWorld 4x4

In [2]:
import numpy as np 
import random 
import tensorflow as tf 
from keras.models import Sequential 
from keras.layers import Dense 
from keras.optimizers import Adam 
from collections import deque

2025-03-27 12:59:53.354676: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-27 12:59:53.394750: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Paramètres du jeu 
GRID_SIZE = 4 
STATE_SIZE = GRID_SIZE * GRID_SIZE 
ACTION_SIZE = 4  # Haut, Bas, Gauche, Droite 
GAMMA = 0.9  # facteur de reduction  
LEARNING_RATE = 0.01 
EPSILON = 1.0  # Exploration initiale 
EPSILON_MIN = 0.01 
EPSILON_DECAY = 0.995 
BATCH_SIZE = 32 
MEMORY_SIZE = 2000 
EPISODES = 1000

In [7]:
# Déplacements possibles (Haut, Bas, Gauche, Droite) 
MOVES = { 
    0: (-1, 0),  # Haut 
    1: (1, 0),   # Bas 
    2: (0, -1),  # Gauche 
    3: (0, 1)    # Droite 
}

In [4]:
class GridWorld: 
    """Environnement GridWorld 4x4""" 
    def __init__(self): 
        self.grid_size = GRID_SIZE 
        self.reset() 
 
    def reset(self): 
        """Réinitialise l'agent à la position de départ.""" 
        self.agent_pos = (0, 0) 
        self.goal_pos = (3, 3) 
        self.obstacle_pos = (1, 1)  # Une case d'obstacle 
        return self.get_state() 
 
    def get_state(self): 
        """Retourne l'état sous forme d'un vecteur binaire.""" 
        state = np.zeros((GRID_SIZE, GRID_SIZE)) 
        state[self.agent_pos] = 1 
        return state.flatten() 
 
    def step(self, action): 
        """Fait avancer l'agent et renvoie (nouvel état, récompense, 
terminé).""" 
        x, y = self.agent_pos 
        dx, dy = MOVES[action] 
        new_x, new_y = x + dx, y + dy 
 
        # Vérifier les limites 
        if 0 <= new_x < GRID_SIZE and 0 <= new_y < GRID_SIZE: 
            self.agent_pos = (new_x, new_y) 
 
        # Vérifier la récompense 
        if self.agent_pos == self.goal_pos: 
            return self.get_state(), 10, True  # Objectif atteint  
        elif self.agent_pos == self.obstacle_pos: 
            return self.get_state(), -5, False  # Obstacle  
        else: 
            return self.get_state(), -1, False  # Déplacement normal

In [5]:
class DQNAgent: 
    """Agent DQN utilisant un réseau de neurones pour apprendre.""" 
    def __init__(self): 
        self.state_size = STATE_SIZE 
        self.action_size = ACTION_SIZE 
        self.memory = deque(maxlen=MEMORY_SIZE) 
        self.epsilon = EPSILON 
        self.model = self._build_model() 
 
    def _build_model(self): 
        """Construit le réseau de neurones.""" 
        model = Sequential([ 
            Dense(24, activation='relu', input_shape=(self.state_size,)), 
            Dense(24, activation='relu'), 
            Dense(self.action_size, activation='linear') 
        ]) 
        model.compile(loss="mse", optimizer=Adam(learning_rate=LEARNING_RATE)) 
        return model 
 
    def remember(self, state, action, reward, next_state, done): 
        """Stocke une expérience dans la mémoire.""" 
        self.memory.append((state, action, reward, next_state, done)) 
 
    def act(self, state): 
        """Choisit une action en suivant une stratégie ε-greedy.""" 
        if np.random.rand() <= self.epsilon: 
            return random.randrange(self.action_size)  # Exploration 
        q_values = self.model.predict(np.array([state]), verbose=0) 
        return np.argmax(q_values[0])  # Exploitation 
 
    def replay(self): 
        """Entraîne le modèle avec des expériences passées.""" 
        if len(self.memory) < BATCH_SIZE: 
            return 
        batch = random.sample(self.memory, BATCH_SIZE) 
        for state, action, reward, next_state, done in batch: 
            target = self.model.predict(np.array([state]), verbose=0)[0] 
            if done: 
                target[action] = reward 
            else: 
                target[action] = reward + GAMMA * np.max(self.model.predict(np.array([next_state]), verbose=0)[0]) 
            self.model.fit(np.array([state]), np.array([target]), epochs=1, verbose=0) 
        if self.epsilon > EPSILON_MIN: 
            self.epsilon *= EPSILON_DECAY  # Réduction de l'exploration

In [8]:
# Entraînement de l'agent 
env = GridWorld() 
agent = DQNAgent() 
 
for episode in range(EPISODES): 
    state = env.reset() 
    total_reward = 0 
    for step in range(50):  # Limite de 50 déplacements 
        action = agent.act(state) 
        next_state, reward, done = env.step(action) 
        agent.remember(state, action, reward, next_state, done) 
        state = next_state 
        total_reward += reward 
        if done: 
            break 
    agent.replay()  # Entraîne le modèle 
    print(f"Épisode {episode+1}/{EPISODES}, Score: {total_reward}, Epsilon: {agent.epsilon:.4f}") 
 
# Sauvegarde du modèle 
agent.model.save("my_model.keras")

Épisode 1/1000, Score: -62, Epsilon: 0.9950
Épisode 2/1000, Score: -25, Epsilon: 0.9900
Épisode 3/1000, Score: -9, Epsilon: 0.9851
Épisode 4/1000, Score: -66, Epsilon: 0.9801
Épisode 5/1000, Score: -1, Epsilon: 0.9752
Épisode 6/1000, Score: -86, Epsilon: 0.9704
Épisode 7/1000, Score: -9, Epsilon: 0.9655
Épisode 8/1000, Score: -9, Epsilon: 0.9607
Épisode 9/1000, Score: -13, Epsilon: 0.9559
Épisode 10/1000, Score: -50, Epsilon: 0.9511
Épisode 11/1000, Score: -6, Epsilon: 0.9464
Épisode 12/1000, Score: -41, Epsilon: 0.9416
Épisode 13/1000, Score: -9, Epsilon: 0.9369
Épisode 14/1000, Score: -27, Epsilon: 0.9322
Épisode 15/1000, Score: -19, Epsilon: 0.9276
Épisode 16/1000, Score: -54, Epsilon: 0.9229
Épisode 17/1000, Score: -19, Epsilon: 0.9183
Épisode 18/1000, Score: -10, Epsilon: 0.9137
Épisode 19/1000, Score: -4, Epsilon: 0.9092
Épisode 20/1000, Score: -18, Epsilon: 0.9046
Épisode 21/1000, Score: -78, Epsilon: 0.9001
Épisode 22/1000, Score: -13, Epsilon: 0.8956
Épisode 23/1000, Score: -1